In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import math
from copy import deepcopy
import xgboost as xgb
from sklearn.svm import SVC
import sklearn.metrics
from copy import deepcopy
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
# stored in dataframe formats
train_dataframe = pd.read_csv('/content/train.csv')
test_dataframe = pd.read_csv('/content/test.csv')

In [ ]:
train_dataframe

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Discontinued
0,1915-IOFGU,Female,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,No,Electronic check,70.50,70.50,Yes
1,6728-CZFEI,Female,0,No,No,15,Yes,No,DSL,No,...,No,No,Yes,No,One year,No,Mailed check,56.15,931.90,No
2,3863-IUBJR,Male,0,Yes,Yes,12,Yes,No,DSL,No,...,No,No,No,Yes,One year,No,Credit card (automatic),53.65,696.35,Yes
3,5572-ZDXHY,Female,0,No,No,22,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,Yes,Mailed check,84.30,1855.65,Yes
4,8348-HFYIV,Male,0,No,No,2,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,49.25,90.35,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,7409-KIUTL,Female,1,No,Yes,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,71.00,71.00,Yes
5339,1937-OTUKY,Female,0,Yes,No,72,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),98.20,7015.90,No
5340,2815-CPTUL,Male,1,No,No,5,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,70.25,331.90,Yes
5341,9805-FILKB,Male,0,Yes,Yes,46,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Bank transfer (automatic),20.20,845.60,No


In [ ]:
print(len(train_dataframe))
print(len(test_dataframe))

5343
1700


In [ ]:
test_dataframe

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0378-NHQXU,Female,0,Yes,Yes,17,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,No,Month-to-month,No,Electronic check,88.25,1460.65
1,4745-LSPLO,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.50,20.50
2,5144-TVGLP,Male,1,No,No,38,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,No,Electronic check,94.75,3653.00
3,3422-GALYP,Male,0,No,No,2,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,79.45,145.15
4,5630-IXDXV,Female,0,No,No,47,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,106.35,4849.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
1696,0396-HUJBP,Female,0,No,No,2,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.30,44.40
1697,3892-NXAZG,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.05,8289.20
1698,3957-SQXML,Female,0,Yes,Yes,34,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.95,894.30


# Data Processing


In [ ]:
"""
This function will transform a unprocessed csv dataframe in
numerical values will can be used for the model. Return type
is also a dataframe type
"""
def preprocess_data(dataframe, labeled):
  # will convert dict to dataframe later
  data = pd.DataFrame()
  # going attribute by attribute
  # ignore customer iD as it is irrelevant
  scaler = StandardScaler()
  data['gender'] = [1 if gender == 'Female' else 0 for gender in dataframe['gender']]
  data['SeniorCitizen'] = [age for age in dataframe['SeniorCitizen']]
  data['Partner'] = [1 if part == 'Yes' else 0 for part in dataframe['Partner']]
  data['Dependents'] = [1 if dep == 'Yes' else 0 for dep in dataframe['Dependents']]
  tenure_reshaped = dataframe['tenure'].values.reshape(-1, 1)
  data['tenure'] = scaler.fit_transform(tenure_reshaped).flatten()
  data['PhoneService'] = [1 if i == 'Yes' else 0 for i in dataframe['PhoneService']]
  data['MultipleLines'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['MultipleLines']]
  data['InternetService'] = [2 if i == 'Fibre Optic' else 1 if i == 'DSL' else 0 for i in dataframe['InternetService']]
  data['OnlineSecurity'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['OnlineSecurity']]
  data['OnlineBackup'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['OnlineBackup']]
  data['DeviceProtection'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['DeviceProtection']]
  data['TechSupport'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['TechSupport']] # Does the customer avail Technical Support
  data['StreamingTV'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['StreamingTV']]
  data['StreamingMovies'] = [2 if i == 'Yes' else 1 if i == 'No' else 0 for i in dataframe['StreamingMovies']]
  data['Contract'] = [2 if i == 'Two year' else 1 if i == 'One Year' else 0 for i in dataframe['Contract']]
  data['PaperlessBilling'] = [1 if i == 'Yes' else 0 for i in dataframe['PaperlessBilling']]
  data['PaymentMethod'] = [3 if i == 'Credit card (automatic)' else 2 if i == 'Bank transfer (automatic)' else 1 if i == 'Electronic Check' else 0 for i in dataframe['PaymentMethod']]
  monthlycharges_reshaped = dataframe['MonthlyCharges'].values.reshape(-1, 1)
  data['MonthlyCharges'] = scaler.fit_transform(monthlycharges_reshaped).flatten()
  totalcharges_reshaped = dataframe['TotalCharges'].values.reshape(-1, 1)
  data['TotalCharges'] = scaler.fit_transform(totalcharges_reshaped).flatten()
  data.fillna(0, inplace=True)
  if labeled:
    data['Discontinued'] = [1 if i == 'Yes' else 0 for i in dataframe['Discontinued']]
    return data.drop('Discontinued', axis=1), data['Discontinued']
  return data

In [ ]:
X_full_train, y_full_train = preprocess_data(train_dataframe, True)
X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
X_test = preprocess_data(test_dataframe, False)
kf = KFold(n_splits=2)

In [ ]:
X_full_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,0,0,-1.283446,1,1,0,1,1,1,1,1,1,0,0,0,0.201289,-0.977142
1,1,0,0,0,-0.712665,1,1,1,1,1,1,1,2,1,0,0,0,-0.276070,-0.595699
2,0,0,1,1,-0.834975,1,1,1,1,1,1,1,1,2,0,0,3,-0.359233,-0.700005
3,1,0,0,0,-0.427274,1,1,0,1,2,1,1,1,2,0,1,0,0.660351,-0.186647
4,0,0,0,0,-1.242676,0,0,1,1,1,2,1,2,2,0,1,0,-0.505601,-0.968352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,1,1,0,1,-1.283446,1,1,0,1,1,1,1,1,1,0,1,0,0.217921,-0.976920
5339,1,0,1,0,1.611233,1,2,0,2,1,2,2,2,1,2,1,2,1.122741,2.098399
5340,0,1,0,0,-1.120366,1,1,0,1,1,1,1,1,1,0,1,0,0.192972,-0.861389
5341,0,0,1,1,0.551209,1,1,0,0,0,0,0,0,0,0,1,2,-1.471962,-0.633914


In [ ]:
# number of nan values in the data
X_test.isna().sum().sum()


0

# Now to create different models and vary their hyperparameters

#Random forest

Grid search with random forest classifier

In [ ]:
# grid search for find best parameters for random forest model
n_estimators = [80, 85, 90, 95, 100, 110]
max_depth = [None, 10, 20]
min_leaf_samples = [1, 2, 3, 4]
bootstraps =[True, False]

best_accuracy = 0
second_best_accuracy = 0
best_parameters = (n_estimators[1], max_depth[1], min_leaf_samples[1], bootstraps[1])
second_best_parameters = (n_estimators[0], max_depth[0], min_leaf_samples[0], bootstraps[0])
iterations = 0
for estimate in n_estimators:
  for depth in max_depth:
    for min_leafs in min_leaf_samples:
      for bootstrap in bootstraps:
        if iterations % 25 == 0:
          print(iterations)
        accuracy = 0
        for i in range(5):
            X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
            clf = RandomForestClassifier(n_estimators=estimate, max_depth=depth, min_samples_leaf=min_leafs, bootstrap=bootstrap)
            clf.fit(X_train, y_train)
            accuracy += sklearn.metrics.roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1]) / 5
        if accuracy > best_accuracy:
          if second_best_accuracy != 0:
            second_best_parameters = deepcopy(best_parameters)
          best_parameters = (estimate, depth, min_leafs, bootstrap)
          best_accuracy = accuracy
        elif accuracy > second_best_accuracy:
          second_best_parameters = (estimate, depth, min_leafs, bootstrap)
          second_best_accuracy = accuracy
        iterations += 1

0


KeyboardInterrupt: 

In [ ]:
print("best accuracy: " + str(best_accuracy))
print(best_parameters)
print("second best accuracy: " + str(best_accuracy))
print(second_best_parameters)

best accuracy: 0.8468239735701644
(95, None, 4, True)
second best accuracy: 0.8468239735701644
(95, 10, 1, False)


final random forest model

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=95, max_depth=None, min_samples_leaf=4, bootstrap=True)
rf_clf.fit(X_full_train, y_full_train)
rf_predictions = rf_clf.predict_proba(X_test)
sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = rf_predictions[:, 1]
sub_df.to_csv('/content/submission_random_forests2.csv', index=False)

#xGboost


xGboost attempt now

In [ ]:
n_estimators = [500]
learning_rates = [0.05, 0.1]
lambdas = [0.01, 0.1, 1]
max_depth = [4, 6]
gamma = [0.01, 0.1, 0.25]
objectives = ['reg:squaredlogerror']

best_accuracy = 0
second_best_accuracy = 0
best_parameters = (n_estimators[0], learning_rates[0], lambdas[0], max_depth[0], gamma[0], objectives[0])
second_best_parameters = (n_estimators[0], learning_rates[0], lambdas[0], max_depth[0], gamma[0], objectives[0])
iterations = 0
for estimate in n_estimators:
  for lr in learning_rates:
    for depth in max_depth:
      for gam in gamma:
        for objective in objectives:
          for lam in lambdas:
            if iterations % 25 == 0:
              print(iterations)
            accuracy = 0
            for i in range(10):
              X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
              clf = xgb.XGBClassifier(n_estimators=estimate, max_depth=depth, learning_rate=lr, gamma=gam, eval_metric='auc', objective=objective, reg_lambda=lam)
              clf.fit(X_train, y_train)
              accuracy += sklearn.metrics.roc_auc_score(y_valid, clf.predict_proba(X_valid)[:,1]) / 10
            if accuracy > best_accuracy:
              if second_best_accuracy != 0:
                second_best_parameters = deepcopy(best_parameters)
              best_parameters = (estimate, lr, lam, depth, gam, objective)
              best_accuracy = accuracy
            elif accuracy > second_best_accuracy:
              second_best_parameters = (estimate, lr, lam, depth, gam, objective)
              second_best_accuracy = accuracy
            iterations += 1

0
25


In [ ]:
print("format: (estimate, lr, lam, depth, gam, objective)")
print("best accuracy: " + str(best_accuracy))
print(best_parameters)
print("second best accuracy: " + str(second_best_accuracy))
print(second_best_parameters)

format: (estimate, lr, lam, depth, gam, objective)
best accuracy: 0.8433721161734797
(500, 0.05, 0.01, 4, 0.1, 'reg:squaredlogerror')
second best accuracy: 0.8414151143190092
(500, 0.05, 0.01, 4, 0.25, 'reg:squaredlogerror')


In [ ]:
# testing various objectives
accuracy = 0
for i in range(10):
  X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
  xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, reg_lambda=0.1, learning_rate=0.05, gamma=0.01, eval_metric='auc')
  xg_clf.fit(X_train, y_train)
  accuracy += sklearn.metrics.roc_auc_score(y_valid, xg_clf.predict_proba(X_valid)[:,1]) / 10
print(accuracy)

accuracy = 0
for i in range(10):
  X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
  xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, reg_lambda=0.1, learning_rate=0.05, gamma=0.01, eval_metric='auc', objective='reg:logistic')
  xg_clf.fit(X_train, y_train)
  accuracy += sklearn.metrics.roc_auc_score(y_valid, xg_clf.predict_proba(X_valid)[:,1]) / 10
print(accuracy)

accuracy = 0
for i in range(10):
  X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
  xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=6, reg_lambda=0.1, learning_rate=0.05, gamma=0.01, eval_metric='auc', objective='reg:squaredlogerror')
  xg_clf.fit(X_train, y_train)
  accuracy += sklearn.metrics.roc_auc_score(y_valid, xg_clf.predict_proba(X_valid)[:,1]) / 10
print(accuracy)

0.8182609255223356
0.8172830268932572
0.8160932852653856


Best xGboost model

In [ ]:
# format: (estimate, lr, lam, depth, gam, objective)
# best accuracy: 0.8433721161734797
# (500, 0.05, 0.01, 4, 0.1, 'reg:squaredlogerror')

# another xgboost attempt
xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=4, reg_lambda=0.01, learning_rate=0.05, gamma=0.1, eval_metric='auc', objective='reg:squaredlogerror')
xg_clf.fit(X_full_train, y_full_train)
xg_predictions = xg_clf.predict_proba(X_test)
sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = xg_predictions[:, 1]
sub_df.to_csv('/content/submission_xgboost5.csv', index=False)

In [ ]:
# xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.08, gamma=0.25, eval_metric='auc')
# xg_clf.fit(X_full_train, y_full_train)
# xg_predictions = xg_clf.predict_proba(X_test)
# sub_df = pd.read_csv('/content/submission.csv')
# sub_df['TARGET'] = xg_predictions[:, 1]
# sub_df.to_csv('/content/submission_xgboost4.csv', index=False)

#Logistic regression

In [ ]:
penalties = ['l1', 'l2', 'elasticnet', None]
c_values = [0.01, 0.05, 0.1, 0.2, 0.5]

best_accuracy = 0
second_best_accuracy = 0
best_parameters = (penalties[1], c_values[1])
second_best_parameters = (penalties[0], c_values[0])
iterations = 0

for penalty in penalties:
  for c in c_values:
    if iterations % 25 == 0:
      print(iterations)
    if penalty == 'l2' or penalty == None:
      # solver depend on the type of penalty
      clf = LogisticRegression(penalty=penalty, C=c, solver='lbfgs')
    elif penalty == 'elasticnet':
      clf = LogisticRegression(penalty=penalty, C=c, solver='saga',l1_ratio=0.5)
    else:
      clf = LogisticRegression(penalty=penalty, C=c, solver='saga')
    clf.fit(X_train, y_train)
    accuracy = sklearn.metrics.roc_auc_score(y_valid, clf.predict_proba(X_valid))
    if accuracy > best_accuracy:
      if second_best_accuracy != 0:
        second_best_parameters = deepcopy(best_parameters)
      best_parameters = (penalty, c)
      best_accuracy = accuracy
    elif accuracy > second_best_accuracy:
      second_best_parameters = (penalty, c)
      second_best_accuracy = accuracy
    iterations += 1

0
c_value 0.01
c_value 0.05


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


c_value 0.1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


c_value 0.2


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


c_value 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ra

c_value 0.01
c_value 0.05
c_value 0.1
c_value 0.2
c_value 0.5


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.

In [ ]:
print("best accuracy: " + str(best_accuracy))
print(best_parameters)
print("second best accuracy: " + str(best_accuracy))
print(second_best_parameters)

best accuracy: 0.8175865294667914
('l2', 0.01)
second best accuracy: 0.8175865294667914
('l2', 0.05)


In [ ]:
# final logistic regression not to be used in ensemble
logistic_clf = LogisticRegression(penalty='l2', C=0.01, solver='lbfgs')
logistic_clf.fit(X_full_train, y_full_train)
logi_predictions = logistic_clf.predict_proba(X_test)
sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = logi_predictions[:, 1]
sub_df.to_csv('/content/submission_logistic1.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


now for adaboost grid search

In [ ]:
base_estimators = [None, DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=5)]
n_estimators = [x for x in range(60, 121, 20)]
learning_rates = [0.2, 0.3, 0.4, 0.5]
algorithms = ['SAMME', 'SAMME.R']

best_accuracy = 0
second_best_accuracy = 0
best_parameters = (base_estimators[1], n_estimators[1], learning_rates[1], algorithms[1])
second_best_parameters = (base_estimators[0], n_estimators[0], learning_rates[0], algorithms[0])
iterations = 0
for estimate in n_estimators:
  for lr in learning_rates:
    for base_estimator in base_estimators:
      for algo in algorithms:
        if iterations % 25 == 0:
          print(iterations)
        accuracy = 0
        for i in range(10):
          X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
          clf = AdaBoostClassifier(estimator=base_estimator, n_estimators=estimate, learning_rate=lr, algorithm=algo)
          clf.fit(X_train, y_train)
          accuracy += sklearn.metrics.roc_auc_score(y_valid, clf.predict_proba(X_valid)[:,1]) / 10
        if accuracy > best_accuracy:
          if second_best_accuracy != 0:
            second_best_parameters = deepcopy(best_parameters)
          best_parameters = (base_estimator, estimate, lr, algo)
          best_accuracy = accuracy
        elif accuracy > second_best_accuracy:
          second_best_parameters = (base_estimator, estimate, lr, algo)
          second_best_accuracy = accuracy
        iterations += 1

0
25
50
75


In [ ]:
print("format: (base_estimator, estimate, lr, algo)")
print("best accuracy: " + str(best_accuracy))
print(best_parameters)
print("second best accuracy: " + str(second_best_accuracy))
print(second_best_parameters)

format: (base_estimator, estimate, lr, algo)
best accuracy: 0.8469343560663539
(DecisionTreeClassifier(max_depth=2), 80, 0.5, 'SAMME')
second best accuracy: 0.8451757527315844
(None, 100, 0.3, 'SAMME')


final adaboost

In [ ]:
# format: (base_estimator, estimate, lr, algo)
# best accuracy: 0.8508214523662749
# (None, 100, 0.3, 'SAMME.R')
# second best accuracy: 0.8467444522175185
# (DecisionTreeClassifier(max_depth=2), 60, 0.3, 'SAMME')
# previous one: estimator=None, n_estimators=60, learning_rate=0.2, algorithm='SAMME.R'

# format: (base_estimator, estimate, lr, algo)
# best accuracy: 0.8469343560663539
# (DecisionTreeClassifier(max_depth=2), 80, 0.5, 'SAMME')
# second best accuracy: 0.8451757527315844
# (None, 100, 0.3, 'SAMME')

ad_clf = AdaBoostClassifier(estimator=None, n_estimators=100, learning_rate=0.3, algorithm='SAMME')
ad_clf.fit(X_full_train, y_full_train)
ad_predictions = ad_clf.predict_proba(X_test)

sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = ad_predictions[:, 1]
sub_df.to_csv('/content/submission_adaboost2.csv', index=False)

Now for SVM

In [ ]:
# grid search for find best parameters for random forest model
Cs = [2.8, 3.0, 3.2]
kernels = ['rbf']
gammas = [0.01, 0.1, 0.2]

best_accuracy = 0
second_best_accuracy = 0
best_parameters = (Cs[0], kernels[0], gammas[0])
second_best_parameters = (Cs[0], kernels[0], gammas[0])
iterations = 0

for C in Cs:
    for kernel in kernels:
        for gamma in gammas:
            if(iterations % 25 == 0):
              print(iterations)
            clf = SVC(probability=True, C=C, kernel=kernel, gamma=gamma, max_iter=10000)
            clf.fit(X_train, y_train)
            accuracy = sklearn.metrics.accuracy_score(y_valid, clf.predict_proba(X_valid))
            if accuracy > best_accuracy:
                if second_best_accuracy != 0:
                    second_best_parameters = deepcopy(best_parameters)
                best_parameters = (C, kernel, gamma)
                best_accuracy = accuracy
            elif accuracy > second_best_accuracy:
                second_best_parameters = (C, kernel, gamma)
                second_best_accuracy = accuracy
            iterations += 1

print(f"Best parameters: {best_parameters} with accuracy: {best_accuracy}")
print(f"Second best parameters: {second_best_parameters} with accuracy: {second_best_accuracy}")

In [ ]:
svm_clf = SVC(probability=True, C=2.8, kernel='rbf', gamma=0.1, max_iter=10000)
svm_clf.fit(X_full_train, y_full_train)
svm_predictions = svm_clf.predict_proba(X_test)

sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = svm_predictions[:, 1]
sub_df.to_csv('/content/submission_svm1.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


#Cat boost

In [ ]:
# create training splits, that wat for the 10 iterations accuracy is most accurate
X_trains = []
X_valids = []
y_trains = []
y_valids = []
for i in range(10):
  X_train, X_valid, y_train, y_valid = train_test_split(X_full_train, y_full_train, test_size=0.2)
  X_trains.append(X_train)
  X_valids.append(X_valid)
  y_trains.append(y_train)
  y_valids.append(y_valid)

In [ ]:
depths = [2, 3, 4]
learning_rate = [0.1, 0.08, 0.05]
iters = [100, 200]
l2_leaf_reg = [4, 5, 6, 7, 8, 9, 13, 15]
colsample_bylevel = [0.6, 0.9]

parameters_score = {}
iterations = 0
for depth in depths:
  for lr in learning_rate:
    for it in iters:
      for l2_reg in l2_leaf_reg:
        for colsample in colsample_bylevel:
          if iterations % 25 == 0:
            print(iterations)
          accuracy = 0
          for i in range(10):
              cat_clf = CatBoostClassifier(verbose=0, eval_metric='AUC', depth=depth, learning_rate=lr, iterations=it, l2_leaf_reg=l2_reg, colsample_bylevel=colsample)
              cat_clf.fit(X_trains[i], y_trains[i])
              accuracy += sklearn.metrics.roc_auc_score(y_valids[i], cat_clf.predict_proba(X_valids[i])[:, 1]) / 10
          parameters_score[(depth, lr, it, l2_reg, colsample)] = accuracy
          iterations += 1

0


In [ ]:
# finds the top ten cat boost parameters
# top 10
# format: (depth, lr, iter, l2_reg, colsample)
# [((3, 0.05, 200, 7, 0.6), 0.8432755807798057),
#  ((2, 0.08, 200, 13, 0.6), 0.8431422891175758),
#  ((2, 0.08, 200, 9, 0.6), 0.8430825379227035),
#  ((3, 0.05, 200, 6, 0.6), 0.8430121437063068),
#  ((3, 0.05, 200, 15, 0.6), 0.8429595322760203),
#  ((3, 0.05, 200, 9, 0.6), 0.8429592389027818),
#  ((3, 0.05, 200, 4, 0.6), 0.8429205519629416),
#  ((3, 0.1, 100, 4, 0.9), 0.8428992727636873),
#  ((4, 0.08, 100, 6, 0.9), 0.842878735650936),
#  ((3, 0.05, 200, 13, 0.6), 0.8428511696146449)]

# vary number of leafs: 4, 7, 9, 13
# vary learning rates 0.05, 0.08
# vary colsample 0.6, 0.9
# vary iterations: 100, 200, 300
print("format: (depth, lr, iter, l2_reg, colsample)")
top_ten = sorted(list(parameters_score.items()), key=lambda item: item[1], reverse=True)[:10]
top_ten

format: (depth, lr, iter, l2_reg, colsample)


[((3, 0.05, 200, 7, 0.6), 0.8432755807798057),
 ((3, 0.05, 300, 7, 0.6), 0.8423004944818826),
 ((3, 0.05, 400, 7, 0.6), 0.8406269909629637),
 ((3, 0.05, 100, 7, 0.6), 0.8402784317166198),
 ((3, 0.05, 1000, 7, 0.6), 0.8336455963853905)]

#Now for ensemble between models

ensemble with many models

In [ ]:
# will combine 6 models

# best xg boost
xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=4, reg_lambda=0.01, learning_rate=0.05, gamma=0.1, eval_metric='auc', objective='reg:squaredlogerror')
xg_clf.fit(X_full_train, y_full_train)
xg_predictions = xg_clf.predict_proba(X_test)

# best adaboost
ad_clf = AdaBoostClassifier(estimator=None, n_estimators=100, learning_rate=0.3, algorithm='SAMME')
ad_clf.fit(X_full_train, y_full_train)
ad_predictions = ad_clf.predict_proba(X_test)

# For the rest of the ensemble will use different versions of catboost that performed best with the testing above
param = {'depth': 4,
         'iterations': 100,
         'learning_rate': 0.1,
         'l2_leaf_reg': 7,
         'colsample_bylevel': 0.9,
         'eval_metric': 'AUC'}
cat_clf1 = CatBoostClassifier(**param)
cat_clf1.fit(X_full_train, y_full_train)
cat_predictions1 = cat_clf1.predict_proba(X_test)

cat_clf2 = CatBoostClassifier(verbose=0, eval_metric='AUC', depth=3, learning_rate=0.05,
                             iterations=200, l2_leaf_reg=7, colsample_bylevel=0.6)
cat_clf2.fit(X_full_train, y_full_train)
cat_predictions2 = cat_clf2.predict_proba(X_test)

cat_clf3 = CatBoostClassifier(verbose=0, eval_metric='AUC', depth=3, learning_rate=0.08,
                             iterations=200, l2_leaf_reg=9, colsample_bylevel=0.6)
cat_clf3.fit(X_full_train, y_full_train)
cat_predictions3 = cat_clf3.predict_proba(X_test)

cat_clf4 = CatBoostClassifier(verbose=0, eval_metric='AUC', depth=2, learning_rate=0.08,
                             iterations=150, l2_leaf_reg=13, colsample_bylevel=0.9)
cat_clf4.fit(X_full_train, y_full_train)
cat_predictions4 = cat_clf4.predict_proba(X_test)

sub_df = pd.read_csv('/content/submission.csv')
zip_predictions = zip(xg_predictions[:, 1], ad_predictions[:, 1], cat_predictions1[:, 1],
                      cat_predictions2[:, 1], cat_predictions3[:, 1], cat_predictions4[:, 1])
predictions = [(xg + ad + cat1 + cat2 + cat3 + cat4) / 6  for xg, ad, cat1, cat2, cat3, cat4 in zip_predictions]
sub_df['TARGET'] = predictions
sub_df.to_csv('/content/submission_final_ensemble1.csv', index=False)

0:	total: 1.69ms	remaining: 167ms
1:	total: 3.12ms	remaining: 153ms
2:	total: 4.59ms	remaining: 148ms
3:	total: 5.92ms	remaining: 142ms
4:	total: 7.35ms	remaining: 140ms
5:	total: 8.78ms	remaining: 138ms
6:	total: 10.2ms	remaining: 136ms
7:	total: 11.9ms	remaining: 137ms
8:	total: 13.5ms	remaining: 136ms
9:	total: 14.9ms	remaining: 135ms
10:	total: 16.6ms	remaining: 134ms
11:	total: 18.2ms	remaining: 133ms
12:	total: 19.5ms	remaining: 130ms
13:	total: 20.8ms	remaining: 128ms
14:	total: 22.4ms	remaining: 127ms
15:	total: 23.7ms	remaining: 124ms
16:	total: 25ms	remaining: 122ms
17:	total: 26.4ms	remaining: 120ms
18:	total: 27.7ms	remaining: 118ms
19:	total: 29.2ms	remaining: 117ms
20:	total: 30.7ms	remaining: 116ms
21:	total: 32.1ms	remaining: 114ms
22:	total: 33.7ms	remaining: 113ms
23:	total: 35.1ms	remaining: 111ms
24:	total: 36.4ms	remaining: 109ms
25:	total: 37.8ms	remaining: 108ms
26:	total: 39.5ms	remaining: 107ms
27:	total: 40.9ms	remaining: 105ms
28:	total: 42.5ms	remaining: 104

In [ ]:
cat_clf2 = CatBoostClassifier(verbose=0, eval_metric='AUC', depth=3, learning_rate=0.05,
                             iterations=200, l2_leaf_reg=7, colsample_bylevel=0.6)
cat_clf2.fit(X_full_train, y_full_train)
cat_predictions2 = cat_clf2.predict_proba(X_test)

sub_df = pd.read_csv('/content/submission.csv')
sub_df['TARGET'] = cat_predictions2[:, 1]
sub_df.to_csv('/content/submission_cat2.csv', index=False)

#another

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=95, max_depth=None, min_samples_leaf=4, bootstrap=True)
rf_clf.fit(X_full_train, y_full_train)
rf_predictions = rf_clf.predict_proba(X_test)

xg_clf = xgb.XGBClassifier(n_estimators=500, max_depth=4, reg_lambda=0.01, learning_rate=0.05, gamma=0.1, eval_metric='auc', objective='reg:squaredlogerror')
xg_clf.fit(X_full_train, y_full_train)
xg_predictions = xg_clf.predict_proba(X_test)

param = {'depth': 4,
         'iterations': 100,
         'learning_rate': 0.1,
         'l2_leaf_reg': 7,
         'colsample_bylevel': 0.9,
         'eval_metric': 'AUC'}
cat_clf = CatBoostClassifier(**param)
cat_clf.fit(X_full_train, y_full_train)
cat_predictions = cat_clf.predict_proba(X_test)

ad_clf = AdaBoostClassifier(estimator=None, n_estimators=100, learning_rate=0.3, algorithm='SAMME')
ad_clf.fit(X_full_train, y_full_train)
ad_predictions = ad_clf.predict_proba(X_test)

sub_df = pd.read_csv('/content/submission.csv')
zip_predictions = zip(xg_predictions[:, 1], rf_predictions[:, 1], ad_predictions[:, 1])
predictions = #Determine after validation scores, rf, logi, ad, svm in zip_predictions]
sub_df['TARGET'] = predictions
sub_df.to_csv('/content/submission_ensemble3.csv', index=False)

# Plots/ Data Analysis

In [ ]:
features = {'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure','PhoneService',
              'MultipleLines','InternetService','OnlineSecurity','OnlineBackup', 'DeviceProtection',
              'TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling',
              'PaymentMethod', 'MonthlyCharges', 'TotalCharges'}
